In [ ]:
# Problem Parameters

max_rooms_per_course = 3 # A course can be alloted to at most 3 rooms
max_courses_per_student_per_day = 3 # A student can have at most 2 courses in a day
weight_minimize_two_exams_per_day = 5.0 # Weight allocated to minimize no. of 2 exams in a day for a student
weight_minimize_rooms_used = 20.0  # Weight allocated to minimize no. of rooms used for a course
weight_professor_preferred_slots = 30.0  # Weight allocated to get preffered slots for professors


In [2]:
! pip install tabulate
! pip install matplotlib
! pip install reportlab
! pip install pypdf2

In [3]:
import gurobipy as gp
from gurobipy import GRB
from IPython.display import display, Math, Latex
from openpyxl import Workbook
import import_ipynb
import data_utils as data
import time
import pandas as pd
from collections import defaultdict
from tabulate import tabulate
from reportlab.lib import colors
from reportlab.lib.pagesizes import LETTER
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import PyPDF2
import os 


tt = gp.Model('IIITB Exams Timetable')

185
65
2165
{0: True,
 1: True,
 2: True,
 3: True,
 4: True,
 5: True,
 6: True,
 7: True,
 8: True,
 9: True,
 10: True,
 11: True,
 12: True,
 13: True,
 14: True,
 15: True,
 16: True,
 17: True,
 18: True,
 19: True,
 20: True,
 21: True,
 22: True,
 23: True,
 24: True,
 25: True,
 26: True,
 27: True,
 28: True,
 29: True,
 30: True,
 31: True,
 32: True,
 33: True,
 34: True,
 35: True,
 36: True,
 37: True,
 38: True,
 39: True,
 40: True,
 41: True,
 42: True,
 43: True,
 44: True,
 45: True,
 46: True,
 47: True,
 48: True,
 49: True,
 50: True,
 51: True,
 52: True}
course_group {0: 'NotLab', 1: 'NotLab', 2: 'NotLab', 3: 'NotLab', 4: 'CSE Lab', 5: 'NotLab', 6: 'NotLab', 7: 'NotLab', 8: 'NotLab', 9: 'NotLab', 10: 'NotLab', 11: 'NotLab', 12: 'NotLab', 13: 'NotLab', 14: 'NotLab', 15: 'NotLab', 16: 'NotLab', 17: 'NotLab', 18: 'NotLab', 19: 'NotLab', 20: 'NotLab', 21: 'NotLab', 22: 'NotLab', 23: 'NotLab', 24: 'CSE Lab', 25: 'NotLab', 26: 'NotLab', 27: 'NotLab', 28: 'NotLab', 29:

In [4]:
def exam_slots(course: int):
    return int(data.get_course_duration(course)//15)

exam_spans = {c: data.begin_end_pairs(exam_slots(c)) for c in range(data.n_courses)}

E = tt.addVars(
    [
        (c ,b ,e, d)
        for c in range(data.n_courses)
        for b, e in exam_spans[c]
        for d in range(data.n_days)
    ],
    vtype=GRB.BINARY,
    name="E",
)

C = tt.addVars(
    [
        
        (c,r)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
    ],
    vtype=GRB.INTEGER,
    lb=0,
    ub=230,
    name="C",
)



## Hard Constraints

In [5]:
def signum(x):
    if x>0:
        return 1
    return 0

def covering_sessions(s:int, t:int):
    # Return the list of begin-end pairs for session 's' overlapping with time 't'
    return [(b,e) for b,e in exam_spans[s] if  data.overlap(b,e,t)]

In [6]:
# Constraint 1: Each exam is scheduled exactly once
tt.addConstrs(
    (E.sum(c, '*', '*', '*') == 1 for c in range(data.n_courses)),
    name="one_exam_per_course"
)

tt.update()

In [7]:
# Constraint 2: Room capacity constraint C[c, r]*E[c, b, e, d] <= room_capacity[r] for all rooms

tt.addConstrs((gp.quicksum(C[c,r]*E[c,b,e,d]
                          for c in range(data.n_courses)
                          for b,e in covering_sessions(c, t)) <= data.get_room_capacity(r)
              for d in range(data.n_days)
              for r in range(data.n_rooms)
              for t in range(data.n_times)),
              name = "room_capacity_constraint"
             )
tt.update()


In [8]:
# Constraint 3: All students for a course should write the exam

tt.addConstrs(
    gp.quicksum(C[c,r] for r in range(data.n_rooms)) == data.get_course_strength(c)
    for c in range(data.n_courses)
)

tt.update()


In [9]:
# Constraint 4: Student clash constraint

tt.addConstrs((gp.quicksum(E[c, b, e, d] for c in data.intersection_pairs[i] for b, e in covering_sessions(c, t)) <= 1
               for i in range(data.n_intersection_pairs)
               for d in range(data.n_days)
               for t in range(data.n_times)),
              name='student_clash_constraint'
             )

tt.update()

In [10]:
# Constraint 5: No course can have more than 3 rooms alloted

M = tt.addVars(
    [
        (c, r)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
    ],
    vtype=GRB.BINARY,
    name = 'M',
)

D1 = tt.addVars(
    [
        (c, r)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
    ],
    vtype=GRB.BINARY,
    name = 'D1',
)

D2 = tt.addVars(
    [
        (c, r)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
    ],
    vtype=GRB.BINARY,
    name = 'D2',
)

for c,r in M:
    room_capacity = data.get_room_capacity(r)
    tt.addConstr(M[c,r] <=1, name = 'M1_constraint')
    tt.addConstr(M[c,r] <=C[c,r], name = 'M2_constraint')
    tt.addConstr(M[c,r] >=D1[c,r], name = 'M3_constraint')
    tt.addConstr(M[c,r] >=C[c,r]-(room_capacity*(1-D2[c,r])), name = 'M4_constraint')
    tt.addConstr(D1[c,r]+D2[c,r] == 1, name = 'M5_constraint')

tt.addConstrs(
    (
        gp.quicksum(M[c, r] for r in range(data.n_rooms)) <= max_rooms_per_course
        for c in range(data.n_courses)
    ),
    name="course_room_constraint",
)

tt.update()

In [11]:
# Constraint 6: Lab exams are conducted in labs

tt.addConstrs(
    (
        gp.quicksum(C[c, r] for r in (data.get_not_rooms_course(c)) )== 0
        for c in range(data.n_courses)
    ),
    name="lab_exam_constraint",
)

tt.update()

In [12]:
# Constraint 7: No triplet courses should be scheduled in the day

tt.addConstrs(
    (
        (E.sum(c1, '*', '*', d) + E.sum(c2, '*', '*', d) + E.sum(c3, '*', '*', d) <= max_courses_per_student_per_day)
        for c1, c2, c3 in data.intersection_triplets
        for d in range(data.n_days)
    ),
    name="triplet_constraint",
)


{(5, 20, 28, 0): <gurobi.Constr *Awaiting Model Update*>,
 (5, 20, 28, 1): <gurobi.Constr *Awaiting Model Update*>,
 (5, 20, 28, 2): <gurobi.Constr *Awaiting Model Update*>,
 (5, 20, 28, 3): <gurobi.Constr *Awaiting Model Update*>,
 (5, 20, 28, 4): <gurobi.Constr *Awaiting Model Update*>,
 (5, 20, 28, 5): <gurobi.Constr *Awaiting Model Update*>,
 (18, 19, 21, 0): <gurobi.Constr *Awaiting Model Update*>,
 (18, 19, 21, 1): <gurobi.Constr *Awaiting Model Update*>,
 (18, 19, 21, 2): <gurobi.Constr *Awaiting Model Update*>,
 (18, 19, 21, 3): <gurobi.Constr *Awaiting Model Update*>,
 (18, 19, 21, 4): <gurobi.Constr *Awaiting Model Update*>,
 (18, 19, 21, 5): <gurobi.Constr *Awaiting Model Update*>,
 (16, 22, 25, 0): <gurobi.Constr *Awaiting Model Update*>,
 (16, 22, 25, 1): <gurobi.Constr *Awaiting Model Update*>,
 (16, 22, 25, 2): <gurobi.Constr *Awaiting Model Update*>,
 (16, 22, 25, 3): <gurobi.Constr *Awaiting Model Update*>,
 (16, 22, 25, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1

## Soft Constraints

In [13]:
# # 1. No 2 exams on a day for a student

# # minimize summation courseintersection(c1, c2) * ((E[c1, *, *, d] + E[c2, *, *, d]) > 1) for all c1, c2, d



X = tt.addVars(
    [
        (c1, c2, d)
        for c1, c2 in data.intersection_pairs
        for d in range(data.n_days)
    ],
    vtype = GRB.BINARY, 
    name = 'X'
)

for c1, c2 in data.intersection_pairs:
    for d in range(data.n_days):
        tt.addConstr(E.sum(c1, '*', '*', d) + E.sum(c2, '*', '*', d) - X[c1, c2, d] <= 1 ,
                        name = 'soft_constraint1'
                    )

## Objective Function

In [14]:
# obj is summation of X[c1, c2, d] * weight(c1, c2) for all c1, c2, d
# for c1, c2 in data.intersection_pairs:
#     for d in range(data.n_days):
#         obj1 += X[c1, c2, d] 
obj1 = gp.quicksum(X[c1, c2, d]*(len(data.get_intersection_of_courses(c1, c2))) for c1, c2 in data.intersection_pairs for d in range(data.n_days))
# for c1, c2 in data.intersection_pairs:
#         print(len(data.get_intersection_of_courses(c1, c2)))
#         print(data.get_course_title(c1), data.get_course_title(c2))
#         print(data.course_student[c1], data.course_student[c2])
#         print(data.get_intersection_of_courses(c1, c2))
#         print('----------------------------------')


In [15]:
# Soft Constraint 2: minimize the number of rooms used
obj2 = gp.quicksum(M[c,r] for c in range(data.n_courses) for r in range(data.n_rooms))



In [16]:
# obj3 = gp.quicksum(E[c, b, e, d] for c in range(data.n_courses) for b, e, d in data.get_preferred_slots(c))
# for c in range(data.n_courses):
#     for b, e, d in data.get_preferred_slots(c):
#         print("c: ", c, "b: ", b, "e: ", e, "d: ", d)
#         obj3 += E[c, b, e, d]

#add veriable Z[c] for each course c. Z[c] = 1 if the course is scheduled in the preferred slot
Z = tt.addVars(
    [
        (c)
        for c in range(data.n_courses)
    ],
    vtype=GRB.BINARY,
    name="Z",
)

tt.addConstrs(
    (Z[c] == gp.quicksum((E[c, b, e, d] for b, e, d in data.get_preferred_slots(c)))
    for c in range(data.n_courses)), 
    name="soft_constraint3"
)

obj3 = E.sum('*', '*', '*', '*')
obj3 -= gp.quicksum(Z[c] for c in range(data.n_courses))
tt.update()

In [17]:
objective =  obj2*weight_minimize_rooms_used + obj3*weight_professor_preferred_slots + obj1*weight_minimize_two_exams_per_day
tt.setObjective(objective, sense=GRB.MINIMIZE)
tt.update()

In [18]:



def cb(model, where):
    if where == GRB.Callback.MIPNODE:
        # Get model objective
        obj = model.cbGet(GRB.Callback.MIPNODE_OBJBST)

        # Has objective changed?
        if abs(obj - model._cur_obj) > 1e-8:
            # If so, update incumbent and time
            model._cur_obj = obj
            model._time = time.time()

    # Terminate if objective has not improved in 60s
    if time.time() - model._time > 90:
        model.terminate()

# tt._cur_obj = float('inf')
# tt._time = time.time()

# tt.optimize(callback=cb)
# # put time limit of 1200 seconds

# run it for 180 sec and dual gap 5 %

tt.setParam('MIPGap', 0.25)
tt.setParam('TimeLimit', 120)
tt.optimize()

Set parameter MIPGap to value 0.25
Set parameter TimeLimit to value 120
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.1 LTS")

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  120
MIPGap  0.25

Academic license 2615578 - for non-commercial use only - registered to sa___@iiitb.ac.in
Optimize a model with 77042 rows, 13873 columns and 858729 nonzeros
Model fingerprint: 0x1a76c66d
Model has 7068 quadratic constraints
Variable types: 0 continuous, 13873 integer (12230 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [5e+00, 8e+02]
  Bounds range     [1e+00, 2e+02]
  RHS range        [1e+00, 2e+02]
  QRHS range       [6e+00, 2e+02]
Presolve removed 70304 rows and 4264 columns
Presolve time: 2.65s
Presolved: 146610 rows, 142689 columns

In [19]:


def clash_report(X_sol, wb):
    wb.append(["Course 1", "Course 2", "Clash"])  
    for (c1, c2) in data.intersection_pairs:
        clash = 0
        for d in range(data.n_days):
            clash += X_sol[c1, c2, d]
        print(c1, c2, clash)
        wb.append([data.course_id_code[c1], data.course_id_code[c2], "True" if clash>=1 else "False"])
        

def exam_slot_report(Esol, Z_sol, wb):
    wb.append(["Course Code", "Title", "size", "CSE/ECE", "day", "duration", "begin", "end", "preferred_time_slot"])

    for c, b, e, d in Esol:
        wb.append([data.course_id_code[c], data.get_course_title(c), data.get_course_strength(c), data.get_dept(c), data.days[d], data.get_course_duration(c), data.times[b], data.times[e], "True" if Z_sol[c] else "False"])

def seating_report(C_sol, Csol, wb):
    wb.append(["Course Code", "Title", "Room", "Room Type", "size", "Room Number", "Capacity", "Seating"])

    for c, r in Csol:
        wb.append([data.course_id_code[c], data.get_course_title(c), data.get_room_num(r), data.get_course_category(c), data.get_course_strength(c), r, data.get_room_capacity(r), C_sol[c, r]])

def make_report(s):
    C_sol = s['C_sol']
    E_sol = s['E_sol']
    M_sol = s['M_sol']
    X_sol = s['X_sol']
    Z_sol = s['Z_sol']
    Esol = []
    for i in E_sol:
        if E_sol[i] > 0:
            Esol.append(i)
    Csol = []
    for i in C_sol:
        if C_sol[i] > 0:
            Csol.append(i)
    wb = Workbook()
    default_sheet = wb.active
    wb.remove(default_sheet)
    clash_sheet = wb.create_sheet(title='Clash Report')
    exam_slot = wb.create_sheet(title='Exam Slot report')
    seating_sheet = wb.create_sheet(title='Seating Report')

    clash_report(X_sol, clash_sheet)
    exam_slot_report(Esol, Z_sol, exam_slot)
    seating_report(C_sol, Csol, seating_sheet)

    save_path = "./timetable_report.xlsx"
    wb.save(save_path)



In [20]:
# extract E C M X values for all best 3 solution counts and print them

def extract_values(model):
    E_sol = model.getAttr('x', E)
    C_sol = model.getAttr('x', C)
    M_sol = model.getAttr('x', M)
    X_sol = model.getAttr('x', X)
    Z_sol = model.getAttr('x', Z)
    return E_sol, C_sol, M_sol, X_sol, Z_sol
# make a 3 pd dataframe for the sol

# E_sol C_sol M_sol X_sol are sets, i want to crerate a dataframe with columns E_sol, C_sol, M_sol, X_sol
def get_bestmax3_solutions(model):
    for i in range(min(1, model.SolCount)):
        s = {}
        model.setParam('SolutionNumber', i)
        E_sol, C_sol, M_sol, X_sol, Z_sol = extract_values(model)
        s['E_sol'] = E_sol
        s['C_sol'] = C_sol
        s['M_sol'] = M_sol
        s['X_sol'] = X_sol
        s['Z_sol'] = Z_sol
        make_report(s)

get_bestmax3_solutions(tt)


16 20 0.0
7 17 0.0
33 36 0.0
10 34 0.0
41 49 0.0
16 29 0.0
44 45 0.0
1 40 0.0
50 52 0.0
48 36 0.0
20 29 0.0
46 48 0.0
5 19 0.0
39 51 0.0
12 34 0.0
11 14 0.0
5 28 0.0
4 30 0.0
40 50 0.0
17 30 0.0
5 37 0.0
9 35 0.0
24 26 0.0
6 20 0.0
16 22 1.0
18 19 1.0
10 36 0.0
41 51 0.0
42 50 0.0
44 47 0.0
1 42 0.0
0 44 0.0
10 45 0.0
20 22 0.0
5 12 0.0
22 28 0.0
2 4 0.0
17 23 0.0
4 23 0.0
12 36 0.0
40 52 0.0
5 30 0.0
12 45 0.0
15 16 0.0
13 28 0.0
25 27 0.0
18 21 0.0
13 37 0.0
42 52 0.0
10 47 0.0
1 44 0.0
11 46 0.0
12 20 0.0
3 17 0.0
31 33 0.0
23 29 0.0
40 45 1.0
5 23 0.0
4 25 0.0
12 47 0.0
27 29 0.0
27 38 0.0
15 18 0.0
16 17 0.0
10 31 0.0
2 27 0.0
42 45 0.0
25 29 0.0
7 23 0.0
18 23 0.0
46 36 0.0
11 48 0.0
20 26 0.0
22 23 0.0
5 7 0.0
29 38 0.0
5 16 0.0
4 18 0.0
31 35 1.0
9 14 0.0
40 47 0.0
3 28 0.0
4 27 0.0
12 31 0.0
19 27 0.0
2 20 0.0
16 19 0.0
7 16 0.0
33 35 0.0
10 33 0.0
2 29 0.0
42 47 0.0
11 32 0.0
16 28 0.0
6 29 0.0
20 28 0.0
4 11 0.0
21 27 0.0
39 41 0.0
3 21 0.0
5 18 0.0
4 20 0.0
22 25 0.0
12 33 

In [21]:
E_sol, C_sol, M_sol, X_sol, Z_sol = extract_values(tt)

solution = []
for i in E_sol:
    if E_sol[i] > 0:
        solution.append(i)

In [22]:
solution_rooms=[]
for i in C_sol:
    if C_sol[i] > 0:
        solution_rooms.append(i)

In [23]:
schedule_by_day = defaultdict(list)

for c, b, e, d in solution:
    rooms = []
    for r in range(data.n_rooms):
        if C_sol[c, r] > 0:
            rooms.append(f"{data.get_room_num(r)} (Capacity: {C_sol[c, r]})")
    entry = {
        "Course": data.course_id_code[c],
        "Strength": data.get_course_strength(c),
        "Time": f"{data.times[b]} - {data.times[e]}",
        "Rooms": ", ".join(rooms),
        "start": data.times[b]
    }
    day = data.days[d]
    schedule_by_day[day].append(entry)

weekdays = ["Mon", "Tue", "Wed", "Thu", "Fri","Sat"]

print("EXAM TIMETABLE")
print("=" * 50)
for day in weekdays:
    if day in schedule_by_day:
        sorted_entries = sorted(
            schedule_by_day[day],
            key=lambda entry: (entry["start"], entry["Strength"])
        )
        for entry in sorted_entries:
            del entry["start"]
        print(f"\n{day}")
        print("-" * 50)
        print(tabulate(sorted_entries, headers="keys", tablefmt="grid"))

EXAM TIMETABLE

Mon
--------------------------------------------------
+-------------+------------+---------------+----------------------------------------------------------------+
| Course      |   Strength | Time          | Rooms                                                          |
+=============+============+===============+================================================================+
| CSE 102-B   |        123 | 09:30 - 11:30 | A106 (Capacity: 123.0)                                         |
+-------------+------------+---------------+----------------------------------------------------------------+
| CSE 824     |        155 | 09:30 - 12:30 | R103 (Capacity: 155.0)                                         |
+-------------+------------+---------------+----------------------------------------------------------------+
| DAS 839     |         72 | 14:00 - 16:00 | A303 (Capacity: 72.0)                                          |
+-------------+------------+---------------+-----

In [24]:
schedule_by_day = defaultdict(list)

for c, b, e, d in solution:
    rooms = []
    for r in range(data.n_rooms):
        if C_sol[c, r] > 0:
            rooms.append(f"{data.get_room_num(r)} (Capacity: {C_sol[c, r]})")
    entry = {
        "Course": data.course_id_code[c],
        "Strength": data.get_course_strength(c),
        "Time": f"{data.times[b]} - {data.times[e]}",
        "Rooms": ", ".join(rooms),
        "start": data.times[b]
    }
    day = data.days[d]
    schedule_by_day[day].append(entry)

weekdays = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat"]


# ----- Save timetable to CSV/Excel -----
rows = []
for day in weekdays:
    if day in schedule_by_day:
        sorted_entries = sorted(
            schedule_by_day[day],
            key=lambda entry: (entry["start"], entry["Strength"])
        )
        for entry in sorted_entries:
            temp_entry = dict(entry)
            temp_entry.pop("start", None)
            temp_entry["Day"] = day
            rows.append(temp_entry)

df = pd.DataFrame(rows)
df.to_csv("exam_timetable.csv", index=False)
df.to_excel("exam_timetable.xlsx", index=False)
print("\nCSV and Excel files have been saved as exam_timetable.csv and exam_timetable.xlsx")

# ----- Save timetable to PDF -----
styles = getSampleStyleSheet()
elements = []
doc = SimpleDocTemplate("exam_timetable.pdf", pagesize=LETTER)
title = Paragraph("EXAM TIMETABLE", styles["Title"])
elements.append(title)
elements.append(Spacer(1, 12))

for day in weekdays:
    if day in schedule_by_day:
        day_header = Paragraph(f"<b>{day}</b>", styles["Heading2"])
        elements.append(day_header)
        elements.append(Spacer(1, 6))
        # Prepare table data.
        sorted_entries = sorted(
            schedule_by_day[day],
            key=lambda entry: (entry["start"], entry["Strength"])
        )
        # Use shallow copies so we don't remove the "start" key from the original.
        pdf_entries = [dict(entry) for entry in sorted_entries]
        for entry in pdf_entries:
            entry.pop("start", None)
        if pdf_entries:
            headers = list(pdf_entries[0].keys())
            table_data = [headers]
            for entry in pdf_entries:
                row_data = [entry[h] for h in headers]
                table_data.append(row_data)
            t = Table(table_data, hAlign="LEFT")
            t.setStyle(TableStyle([
                ("BACKGROUND", (0, 0), (-1, 0), colors.grey),
                ("TEXTCOLOR", (0, 0), (-1, 0), colors.whitesmoke),
                ("ALIGN", (0, 0), (-1, -1), "LEFT"),
                ("GRID", (0,0), (-1,-1), 1, colors.black),
                ("FONTNAME", (0, 0), (-1, 0), "Helvetica-Bold")
            ]))
            elements.append(t)
            elements.append(Spacer(1, 12))

doc.build(elements)
print("PDF file has been saved as exam_timetable.pdf")


CSV and Excel files have been saved as exam_timetable.csv and exam_timetable.xlsx
PDF file has been saved as exam_timetable.pdf


In [25]:
from collections import defaultdict
from tabulate import tabulate

# Turn interactive mode off.
plt.ioff()

# Helper: convert a time string "HH:MM" to minutes since midnight.
def time_to_minutes(timestr):
    h, m = map(int, timestr.split(':'))
    return h * 60 + m

# --- Build the textual timetable (from your provided code) ---
schedule_by_day = defaultdict(list)

for c, b, e, d in solution:
    rooms = []
    for r in range(data.n_rooms):
        if C_sol[c, r] > 0:
            rooms.append(f"{data.get_room_num(r)} (Capacity: {C_sol[c, r]})")
    entry = {
        "Course": data.course_id_code[c],
        "Strength": data.get_course_strength(c),
        "Time": f"{data.times[b]} - {data.times[e]}",
        "Rooms": ", ".join(rooms),
        "start": data.times[b]  # used for sorting only
    }
    day = data.days[d]
    schedule_by_day[day].append(entry)

weekdays = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat"]

# --- Create the visualization for all days on one page ---
days_with_exams = [day for day in weekdays if day in schedule_by_day]
n_days = len(days_with_exams)

fig, axes = plt.subplots(n_days, 1, figsize=(10, 2.5 * n_days), sharex=True)
if n_days == 1:
    axes = [axes]

# Colors for exam blocks.
colors = plt.cm.tab10.colors

for ax, day in zip(axes, days_with_exams):
    exams = sorted(schedule_by_day[day], key=lambda x: time_to_minutes(x["Time"].split(" - ")[0]))
    for idx, exam in enumerate(exams):
        start_str, end_str = exam["Time"].split(" - ")
        start_min = time_to_minutes(start_str)
        end_min = time_to_minutes(end_str)
        duration = end_min - start_min

        rect = mpatches.Rectangle((start_min, idx - 0.4), duration, 0.8,
                                  color=colors[idx % len(colors)], alpha=0.8)
        ax.add_patch(rect)
        label = f"{exam['Course']}\n(Str: {exam['Strength']})"
        ax.text(start_min + duration/2, idx, label, ha="center", va="center", fontsize=9, color="black")
    
    ax.set_ylim(-1, len(exams))
    ax.set_yticks(range(len(exams)))
    ytick_labels = [exam["Rooms"] + " | Str: " + str(exam["Strength"]) for exam in exams]
    ax.set_yticklabels(ytick_labels)
    ax.set_title(day)
    ax.grid(True, axis="x", linestyle="--", alpha=0.6)

all_start = min(time_to_minutes(exam["Time"].split(" - ")[0])
                for day in days_with_exams 
                for exam in schedule_by_day[day])
all_end = max(time_to_minutes(exam["Time"].split(" - ")[1])
              for day in days_with_exams 
              for exam in schedule_by_day[day])
ax_pad = 30
plt.xlim(all_start - ax_pad, all_end + ax_pad)

xticks = list(range((all_start - ax_pad) // 60 * 60, (all_end + ax_pad) + 1, 60))
xtick_labels = [f"{t//60:02d}:{t % 60:02d}" for t in xticks]
plt.xticks(xticks, xtick_labels, rotation=45)
plt.xlabel("Time")

plt.savefig("exam_timetable_plot.pdf")
plt.close(fig)  # close the figure to avoid output

print("Visualization saved as exam_timetable_plot.pdf")

# --- Create separate visualization for each day (wider plots) ---
for day in days_with_exams:
    exams = sorted(schedule_by_day[day], key=lambda x: time_to_minutes(x["Time"].split(" - ")[0]))
    n_exams = len(exams)
    # Increase the width to 16 inches (adjust as needed) and scale the height based on number of exams.
    fig, ax = plt.subplots(figsize=(16, max(4, n_exams * 1.5)))
    
    colors = plt.cm.tab10.colors

    for idx, exam in enumerate(exams):
        start_str, end_str = exam["Time"].split(" - ")
        start_min = time_to_minutes(start_str)
        end_min = time_to_minutes(end_str)
        duration = end_min - start_min

        rect = mpatches.Rectangle((start_min, idx - 0.4), duration, 0.8,
                                  color=colors[idx % len(colors)], alpha=0.8)
        ax.add_patch(rect)
        label = f"{exam['Course']}\n(Str: {exam['Strength']})"
        ax.text(start_min + duration/2, idx, label, ha="center", va="center", fontsize=9, color="black")
    
    ax.set_ylim(-1, n_exams)
    ax.set_yticks(range(n_exams))
    ytick_labels = [exam["Rooms"] + " | Str: " + str(exam["Strength"]) for exam in exams]
    ax.set_yticklabels(ytick_labels)
    ax.set_title(day)
    ax.grid(True, axis="x", linestyle="--", alpha=0.6)

    all_start = min(time_to_minutes(exam["Time"].split(" - ")[0]) for exam in exams)
    all_end = max(time_to_minutes(exam["Time"].split(" - ")[1]) for exam in exams)
    pad = 30
    ax.set_xlim(all_start - pad, all_end + pad)
    xticks = list(range((all_start - pad) // 60 * 60, all_end + pad + 1, 60))
    xtick_labels = [f"{t//60:02d}:{t % 60:02d}" for t in xticks]
    ax.set_xticks(xticks)
    ax.set_xticklabels(xtick_labels, rotation=45)
    ax.set_xlabel("Time")
    
    fig.savefig(f"exam_timetable_plot_{day}.pdf")
    plt.close(fig)  # close the figure

print("Separate visualizations saved as exam_timetable_plot_{day}.pdf files for each day")

Visualization saved as exam_timetable_plot.pdf
Separate visualizations saved as exam_timetable_plot_{day}.pdf files for each day


In [26]:

def merge_pdfs(pdf_list, output_filename):
    merger = PyPDF2.PdfMerger()
    
    for pdf in pdf_list:
        merger.append(pdf)

    merger.write(output_filename)
    merger.close()
    print(f"Merged PDF saved as {output_filename}")

# List of input PDFs (ensure correct file paths)
pdf_files = [
    "exam_timetable_plot_Mon.pdf",
    "exam_timetable_plot_Tue.pdf",
    "exam_timetable_plot_Wed.pdf",
    "exam_timetable_plot_Thu.pdf",
    "exam_timetable_plot_Fri.pdf",
    "exam_timetable_plot_Sat.pdf"
]
output_pdf = "exam_timetable_plot.pdf"



merge_pdfs(pdf_files, output_pdf)

# Delete all the pdf_files
for pdf in pdf_files:
    os.remove(pdf)


Merged PDF saved as exam_timetable_plot.pdf


In [27]:
## Verification of the solution

# 1 - No student should have two exams at the same time

def time_to_minutes(time_str):
    """Converts a HH:MM string to minutes since midnight."""
    hh, mm = map(int, time_str.split(":"))
    return hh * 60 + mm

# Build a dictionary that maps each course (c) to its one exam slot details.
# Assume each course appears only once in the solution.
course_slots = {}
for c, b, e, d in solution:
    # Save day and exam start/end times (converted to minutes) for each course
    day = data.days[d]
    start = time_to_minutes(data.times[b])
    end   = time_to_minutes(data.times[e])
    course_slots[c] = (day, start, end)

# Check for any student conflicts between every pair of distinct courses.
clash_found = False
course_indices = list(course_slots.keys())
for i in range(len(course_indices)):
    c1 = course_indices[i]
    day1, start1, end1 = course_slots[c1]
    for j in range(i + 1, len(course_indices)):
        c2 = course_indices[j]
        day2, start2, end2 = course_slots[c2]
        # Check if courses are on the same day and if their times overlap:
        if day1 == day2 and (start1 < end2 and start2 < end1):
            # Check intersection of enrolled students; get_intersection_of_courses returns set of common student IDs.
            common_students = set(data.course_student[c1]) & set(data.course_student[c2])
            if common_students:
                clash_found = True
                print(f"Clash found between Course {data.course_id_code[c1]} and Course {data.course_id_code[c2]}")
                print("Common student(s):", common_students)
                print(f"--> {data.course_id_code[c1]}: {data.times[b]} - {data.times[e]} ;",  # later you can format times properly
                      f"{data.course_id_code[c2]}: {data.times[b]} - {data.times[e]}\n")

if not clash_found:
    print("No student clashes found!")


# 2 - The total number of students writing an exam in a room should not exceed the room capacity


room_violations = False

# Process each room separately
for r in range(data.n_rooms):
    # Build a schedule per day for room r.
    room_schedule = defaultdict(list)
    for c, b, e, d in solution:
        allocated = C_sol[c, r]
        if allocated > 0:
            day = data.days[d]
            start = time_to_minutes(data.times[b])
            end = time_to_minutes(data.times[e])
            course_code = data.course_id_code[c]
            room_schedule[day].append((start, end, allocated, course_code))
    
    # For each day, process the exam intervals for room r.
    for day, intervals in room_schedule.items():
        events = []
        for start, end, alloc, course in intervals:
            # When exam starts, add the number of students;
            # when exam ends, subtract that number.
            events.append((start, alloc))
            events.append((end, -alloc))
        events.sort(key=lambda x: x[0])
        
        current_alloc = 0
        for time, change in events:
            current_alloc += change
            if current_alloc > data.get_room_capacity(r):
                room_violations = True
                print(f"Room capacity violation on {day} in room {data.get_room_num(r)}:")
                print(f"   Room capacity: {data.room_capacity[r]}, allocated: {current_alloc}")
                break  # Report once per day per room

if not room_violations:
    print("No room capacity violations found!")


No student clashes found!
No room capacity violations found!


In [28]:
for s in range (data.n_students):
    l=[]
    course_list=data.get_courses_student(s)
    for c,b,e,d in solution:
        if E_sol[c,b,e,d] == 1 and c in course_list:
            l.append((data.course_id_code[c], data.times[b], data.times[e], data.days[d]))
    
    print(f"Student {s+1} has exams for courses {l}")

Student 1 has exams for courses [('AIM 101', '10:45', '12:45', 'Wed'), ('AIM 102', '15:00', '18:00', 'Tue'), ('CSE 102-B', '09:30', '11:30', 'Mon'), ('CSE 102 P-B', '10:15', '12:45', 'Thu'), ('CSE 104-B', '14:00', '17:00', 'Fri'), ('DAS 101-B', '14:15', '17:15', 'Mon'), ('DAS 101P-B', '14:45', '17:15', 'Sat')]
Student 2 has exams for courses [('AIM 101', '10:45', '12:45', 'Wed'), ('AIM 102', '15:00', '18:00', 'Tue'), ('CSE 102-B', '09:30', '11:30', 'Mon'), ('CSE 102 P-B', '10:15', '12:45', 'Thu'), ('CSE 104-B', '14:00', '17:00', 'Fri'), ('DAS 101-B', '14:15', '17:15', 'Mon'), ('DAS 101P-B', '14:45', '17:15', 'Sat')]
Student 3 has exams for courses [('AIM 101', '10:45', '12:45', 'Wed'), ('AIM 102', '15:00', '18:00', 'Tue'), ('CSE 102-B', '09:30', '11:30', 'Mon'), ('CSE 102 P-B', '10:15', '12:45', 'Thu'), ('CSE 104-B', '14:00', '17:00', 'Fri'), ('DAS 101-B', '14:15', '17:15', 'Mon'), ('DAS 101P-B', '14:45', '17:15', 'Sat')]
Student 4 has exams for courses [('AIM 101', '10:45', '12:45', '

In [29]:
for s in range(data.n_students):
    print(f"Student {s+1} has exam on: " , [data.days[d] for c,b,e,d in solution if E_sol[c,b,e,d] == 1 and c in data.get_courses_student(s)])

Student 1 has exam on:  ['Wed', 'Tue', 'Mon', 'Thu', 'Fri', 'Mon', 'Sat']
Student 2 has exam on:  ['Wed', 'Tue', 'Mon', 'Thu', 'Fri', 'Mon', 'Sat']
Student 3 has exam on:  ['Wed', 'Tue', 'Mon', 'Thu', 'Fri', 'Mon', 'Sat']
Student 4 has exam on:  ['Wed', 'Tue', 'Mon', 'Thu', 'Fri', 'Mon', 'Sat']
Student 5 has exam on:  ['Wed', 'Tue', 'Mon', 'Thu', 'Fri', 'Mon', 'Sat']
Student 6 has exam on:  ['Wed', 'Tue', 'Mon', 'Thu', 'Fri', 'Mon', 'Sat']
Student 7 has exam on:  ['Wed', 'Tue', 'Mon', 'Thu', 'Fri', 'Mon', 'Sat']
Student 8 has exam on:  ['Wed', 'Tue', 'Mon', 'Thu', 'Fri', 'Mon', 'Sat']
Student 9 has exam on:  ['Wed', 'Tue', 'Mon', 'Thu', 'Fri', 'Mon', 'Sat']
Student 10 has exam on:  ['Wed', 'Tue', 'Mon', 'Thu', 'Fri', 'Mon', 'Sat']
Student 11 has exam on:  ['Wed', 'Tue', 'Mon', 'Thu', 'Fri', 'Mon', 'Sat']
Student 12 has exam on:  ['Wed', 'Tue', 'Mon', 'Thu', 'Fri', 'Mon', 'Sat']
Student 13 has exam on:  ['Wed', 'Tue', 'Mon', 'Thu', 'Fri', 'Mon', 'Sat']
Student 14 has exam on:  ['Wed', '

In [30]:
# print how many courses have got the preferred slots
for c in range(data.n_courses):
    print(f"Course {data.course_id_code[c]} has got the preferred slots: ", [data.times[b] for b,e,d in data.get_preferred_slots(c) if E_sol[c,b,e,d] == 1])

Course AIM 101 has got the preferred slots:  ['10:45']
Course AIM 102 has got the preferred slots:  ['15:00']
Course AIM 704 has got the preferred slots:  ['14:45']
Course AIM 821 has got the preferred slots:  ['09:15']
Course AIM 831 has got the preferred slots:  ['09:45']
Course AIM 832 has got the preferred slots:  ['14:00']
Course AIM 843 has got the preferred slots:  ['14:00']
Course AIM 845 has got the preferred slots:  ['15:00']
Course AIM 846 has got the preferred slots:  ['14:15']
Course AMS 401 has got the preferred slots:  ['09:30']
Course AMS 402 has got the preferred slots:  ['14:00']
Course AMS 403 has got the preferred slots:  ['10:15']
Course AMS 404 has got the preferred slots:  ['09:30']
Course COM 605 has got the preferred slots:  ['10:30']
Course CSE 212 has got the preferred slots:  ['14:15']
Course CSE 606 has got the preferred slots:  ['14:00']
Course CSE 704 has got the preferred slots:  ['14:45']
Course CSE 718 has got the preferred slots:  ['14:15']
Course CSE

In [31]:
for c1, c2 in data.intersection_pairs:
    clash = 0
    for d in range(data.n_days):
        clash += X_sol[c1, c2, d]
        if(X_sol[c1, c2, d] > 0):
            print(data.course_id_code[c1], data.course_id_code[c2], d)
            
    print(data.course_id_code[c1], data.course_id_code[c2], clash>0)

CSE 704 CSE 824 False
AIM 845 CSE 718 False
ECE 303 EGC 301 False
AMS 402 ECE 303P False
CSE 102 P-A EGC 121-A False
CSE 704 DHS 315 False
CSE 104-B DAS 101-B False
AIM 102 CSE 102-B False
EGC 121-B EGC 123-B False
DAS 101P-A EGC 301 False
CSE 824 DHS 315 False
DAS 101-A DAS 101P-A False
AIM 832 CSE 816 False
CSE 102-A EGC 123-A False
AMS 404 ECE 303P False
AMS 403 CSE 212 False
AIM 832 DHS 314 False
AIM 831 DHS 611 False
CSE 102-B EGC 121-B False
CSE 718 DHS 611 False
AIM 832 VLS 603 False
AMS 401 ECE 304 False
DHS 108 DHS 306 False
AIM 843 CSE 824 False
CSE 704 DAS 605 4
CSE 704 DAS 605 True
CSE 754 CSE 816 1
CSE 754 CSE 816 True
AMS 402 EGC 301 False
CSE 102 P-A EGC 123-A False
CSE 102 P-B EGC 121-B False
CSE 104-B DAS 101P-B False
AIM 102 CSE 102 P-B False
AIM 101 CSE 104-B False
AMS 402 DAS 101-B False
CSE 824 DAS 605 False
AIM 832 AMS 404 False
DAS 605 DHS 314 False
AIM 704 AIM 831 False
CSE 718 DAS 839 False
AIM 831 DAS 839 False
AMS 404 EGC 301 False
CSE 102-B EGC 123-B False
A

In [32]:
# print all X(c1, c2, d) that are non zeros
for c1, c2 in data.intersection_pairs:
    for d in range(data.n_days):
        if X_sol[c1, c2, d] == 0:
            print(data.course_id_code[c1], data.course_id_code[c2], d)
        # print sum of E[c1, *, *, d] + E[c2, *, *, d] for that d
        Ec1d = 0
        for b, e in exam_spans[c1]:
            Ec1d += E_sol[c1, b, e, d]
        Ec2d = 0
        for b, e in exam_spans[c2]:
            Ec2d += E_sol[c2, b, e, d]
        print(data.course_id_code[c1], data.course_id_code[c2], d, Ec1d, Ec2d)
        print(X_sol[c1, c2, d])

CSE 704 CSE 824 0
CSE 704 CSE 824 0 0.0 1.0
-0.0
CSE 704 CSE 824 1
CSE 704 CSE 824 1 0.0 0.0
-0.0
CSE 704 CSE 824 2
CSE 704 CSE 824 2 0.0 0.0
-0.0
CSE 704 CSE 824 3
CSE 704 CSE 824 3 0.0 0.0
-0.0
CSE 704 CSE 824 4
CSE 704 CSE 824 4 1.0 0.0
-0.0
CSE 704 CSE 824 5
CSE 704 CSE 824 5 0.0 0.0
-0.0
AIM 845 CSE 718 0
AIM 845 CSE 718 0 0.0 0.0
-0.0
AIM 845 CSE 718 1
AIM 845 CSE 718 1 0.0 0.0
-0.0
AIM 845 CSE 718 2
AIM 845 CSE 718 2 0.0 1.0
-0.0
AIM 845 CSE 718 3
AIM 845 CSE 718 3 0.0 0.0
-0.0
AIM 845 CSE 718 4
AIM 845 CSE 718 4 0.0 0.0
-0.0
AIM 845 CSE 718 5
AIM 845 CSE 718 5 1.0 0.0
-0.0
ECE 303 EGC 301 0
ECE 303 EGC 301 0 0.0 0.0
-0.0
ECE 303 EGC 301 1
ECE 303 EGC 301 1 0.0 0.0
-0.0
ECE 303 EGC 301 2
ECE 303 EGC 301 2 0.0 1.0
-0.0
ECE 303 EGC 301 3
ECE 303 EGC 301 3 1.0 0.0
-0.0
ECE 303 EGC 301 4
ECE 303 EGC 301 4 0.0 0.0
-0.0
ECE 303 EGC 301 5
ECE 303 EGC 301 5 0.0 0.0
-0.0
AMS 402 ECE 303P 0
AMS 402 ECE 303P 0 0.0 0.0
-0.0
AMS 402 ECE 303P 1
AMS 402 ECE 303P 1 1.0 0.0
-0.0
AMS 402 ECE 303P

In [33]:
print(solution)

[(0, 11, 19, 2), (1, 25, 37, 1), (2, 24, 36, 2), (3, 5, 17, 3), (4, 7, 19, 5), (5, 21, 33, 3), (6, 21, 25, 2), (7, 25, 37, 5), (8, 22, 34, 5), (9, 6, 18, 1), (10, 21, 33, 1), (11, 9, 19, 1), (12, 6, 14, 1), (13, 10, 18, 4), (14, 22, 30, 3), (15, 21, 27, 3), (16, 24, 34, 4), (17, 22, 34, 2), (18, 21, 33, 1), (19, 10, 18, 1), (20, 6, 18, 0), (21, 7, 19, 2), (22, 11, 19, 4), (23, 21, 29, 0), (24, 22, 34, 5), (25, 22, 30, 0), (26, 29, 37, 3), (27, 5, 17, 4), (28, 11, 19, 2), (29, 21, 29, 1), (30, 24, 36, 0), (31, 22, 34, 4), (32, 25, 37, 0), (33, 5, 17, 3), (34, 7, 19, 5), (35, 6, 12, 4), (36, 21, 33, 2), (37, 21, 33, 1), (38, 7, 19, 2), (39, 7, 19, 5), (40, 6, 14, 0), (41, 25, 35, 0), (42, 9, 19, 3), (43, 7, 15, 4), (44, 21, 33, 4), (45, 22, 34, 0), (46, 24, 36, 5), (47, 24, 34, 5), (48, 24, 34, 4), (49, 6, 18, 2), (50, 21, 33, 1), (51, 26, 34, 1), (52, 26, 34, 2)]


In [34]:
count = 0
for c1, c2 in data.intersection_pairs:
    for d in range(data.n_days):
        se=0
        for b, e in exam_spans[c1]:
            se += E_sol[c1, b, e, d] 
        for b, e in exam_spans[c2]:
            se += E_sol[c2, b, e, d]
        if(se > 1):
            count+=1
            print(f'{len(data.get_intersection_of_courses(c1,c2))},X: {X_sol[c1, c2, d]}')

print(count)

7,X: 1.0
6,X: 1.0
59,X: 1.0
61,X: 1.0
13,X: 1.0
1,X: 1.0
3,X: 1.0
2,X: 1.0
2,X: 1.0
9


In [35]:
len(data.intersection_pairs)

282